# LAB 3 - UART integration

In [27]:
import serial.tools.list_ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p)

/dev/ttyUSB0 - USB Serial
/dev/ttyUSB1 - TUL - TUL


In [34]:
import serial
ser = serial.Serial('/dev/ttyUSB0', 115200, timeout=10)

First, create two random operands (A,B) that have OPERAND_WIDTH length.

In [1]:
import random
import time

OPERAND_WIDTH = 512

N_BYTES = int(OPERAND_WIDTH/8)

A = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)
B = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)

if A < B:
    A, B = B, A

res = A + B

print("A     = ", hex(A))
print("B     = ", hex(B))
print("A + B = ", hex(res))

A_bytes = bytearray.fromhex(format(A, 'x'))
B_bytes = bytearray.fromhex(format(B, 'x'))

A     =  0xf3dde4d25f1a5a7151c2a0abf30c02ae17e2f3745699ee09f670b5f3253ddd219cec739307d903dfdd67584adfe92a1a6128067abf97d5abf66756eb6371a0ee
B     =  0xc1dd392350802a1f50f8b40cce3b50dd257e870fa0d8c111ebd391d368db703a2bc5fca830fcdc573f5e13f843738ec1d961cbc7496f0f762d2fe6d310a1ba95
A + B =  0x1b5bb1df5af9a8490a2bb54b8c147538b3d617a83f772af1be24447c68e194d5bc8b2703b38d5e0371cc56c43235cb8dc3a89d2420906e52223973dbe74135b83


Then, send the two operands (A,B) via the serial port, byte by byte. Later, the PYNQ board receives the result of the addition, and transmits it back to the computer.

In [38]:
for op_byte in A_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

for op_byte in B_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

res_rcvd = ser.read(N_BYTES+1) 

res2 = int.from_bytes(res_rcvd, "big")

if res2==res:
    print("Result received correctly!")
    print("Expected   = ", hex(res))
    print("Received   = ", hex(res2))
else:
    print("Result INCORRECT!")
    print("Expected   = ", hex(res))
    print("Received   = ", hex(res2))
    print("Offsert:   = ", hex(res2-res))


Result received correctly!
Expected   =  0x19bac79653e722eeaf30958a840a36dbb71b3c46d967f075ba2871383a710414cf5fea190cf42a403eb49d274fd4879aa7f415845a4ac9ddd72e94a919e50213e
Received   =  0x19bac79653e722eeaf30958a840a36dbb71b3c46d967f075ba2871383a710414cf5fea190cf42a403eb49d274fd4879aa7f415845a4ac9ddd72e94a919e50213e


In [31]:
ser.close()